In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pandas as pd
import numpy as np  # To handle missing values with NaN

# Local variable to map league URLs to league names
league_urls = [
    ( "England Premier League",  "https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1"),
    ( "Moldova Super Liga",  "https://www.transfermarkt.com/super-liga/startseite/wettbewerb/MO1N"),
    ( "Spain La Liga","https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1" ),
    ("Italy Serie A","https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1" ),
    ("Germany Bundesliga","https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1"), 
    ("France Ligue 1", "https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1" ),
    ("Portugal Liga Portugal","https://www.transfermarkt.com/liga-portugal/startseite/wettbewerb/PO1"), 
    ("Netherlands Eredivise", "https://www.transfermarkt.com/eredivisie/startseite/wettbewerb/NL1"),
    ("Turkey Super Lig", "https://www.transfermarkt.com/super-lig/startseite/wettbewerb/TR1"),
    ("Russia Premier Liga", "https://www.transfermarkt.com/premier-liga/startseite/wettbewerb/RU1"),
    ("Belgium Jupiler Pro League","https://www.transfermarkt.com/jupiler-pro-league/startseite/wettbewerb/BE1"), 
    ("Greece Super League 1", "https://www.transfermarkt.com/super-league-1/startseite/wettbewerb/GR1"),
    ("Austria Bundesliga", "https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/A1"),
    ("Ukraine Premier Liga", "https://www.transfermarkt.com/premier-liga/startseite/wettbewerb/UKR1"),
    ("Switzerland Super League", "https://www.transfermarkt.com/super-league/startseite/wettbewerb/C1"),
    ("Denmark Superliga", "https://www.transfermarkt.com/superliga/startseite/wettbewerb/DK1"),
    ("Czech Republic Chance Liga","https://www.transfermarkt.com/chance-liga/startseite/wettbewerb/TS1"), 
    ("Scotland Premiership", "https://www.transfermarkt.com/scottish-premiership/startseite/wettbewerb/SC1"),
    ("Serbia Super liga Srbije", "https://www.transfermarkt.com/super-liga-srbije/startseite/wettbewerb/SER1"),
    ("Poland Ekstraklasa","https://www.transfermarkt.com/pko-bp-ekstraklasa/startseite/wettbewerb/PL1"),
    ("Croatia SuperSport HNL", "https://www.transfermarkt.com/supersport-hnl/startseite/wettbewerb/KR1"),
    ("Sweden Allsvenskan", "https://www.transfermarkt.com/allsvenskan/startseite/wettbewerb/SE1"),
    ("Norway Eliteserien", "https://www.transfermarkt.com/eliteserien/startseite/wettbewerb/NO1"),
    ("Romania SuperLiga", "https://www.transfermarkt.com/superliga/startseite/wettbewerb/RO1"),
    ("Bulgaria efbet Liga", "https://www.transfermarkt.com/efbet-liga/startseite/wettbewerb/BU1"),
    ("Hungary NB I.", "https://www.transfermarkt.com/nemzeti-bajnoksag/startseite/wettbewerb/UNG1"),
    ("Cyprus Protahlima Cyta","https://www.transfermarkt.com/protathlima-cyta/startseite/wettbewerb/ZYP1"), 
    ("Israel Ligat ha'Al", "https://www.transfermarkt.com/ligat-haal/startseite/wettbewerb/ISR1"),
    ("Slovakia Nike Liga", "https://www.transfermarkt.com/nike-liga/startseite/wettbewerb/SLO1"),
    ("Azerbaijan Premyer Liqa", "https://www.transfermarkt.com/premyer-liqa/startseite/wettbewerb/AZ1"),
    ("Khazakstan Premier Liga", "https://www.transfermarkt.com/premier-liga/startseite/wettbewerb/KAS1"),
    ("Bosnia-Herzegovina Premijer Liga BiH","https://www.transfermarkt.com/premijer-liga-bosne-i-hercegovine/startseite/wettbewerb/BOS1"), 
    ("Belarus Vysheyshaya Liga", "https://www.transfermarkt.com/vysheyshaya-liga/startseite/wettbewerb/WER1"),
    ("Slovenia Prva Liga", "https://www.transfermarkt.com/prva-liga/startseite/wettbewerb/SL1"),
    ("Lithuania A Lyga", "https://www.transfermarkt.com/a-lyga/startseite/wettbewerb/LI1"),
    ("Finland Veikkausliiga", "https://www.transfermarkt.com/veikkausliiga/startseite/wettbewerb/FI1"),
    ("Latvia Virsliga", "https://www.transfermarkt.com/virsliga/startseite/wettbewerb/LET1"),
    ("Armenia Bardzraguyn khumb","https://www.transfermarkt.com/bardzraguyn-khumb/startseite/wettbewerb/ARM1"), 
    ("Albania Kategoria Superiore", "https://www.transfermarkt.com/kategoria-superiore/startseite/wettbewerb/ALB1"),
    ("North Macedonia Prva liga", "https://www.transfermarkt.com/prva-makedonska-fudbalska-liga/startseite/wettbewerb/MAZ1"),
    ("Georgia Erovnuli Liga", "https://www.transfermarkt.com/erovnuli-liga/startseite/wettbewerb/GE1N"),
    ("Kosovo Superliga e Kosovës", "https://www.transfermarkt.com/superliga-e-kosoves/startseite/wettbewerb/KO1"),
    ("Malta Premier League Opening Round","https://www.transfermarkt.com/premier-league-opening-round/startseite/wettbewerb/MT1N"), 
    ("Ireland Premier Division", "https://www.transfermarkt.com/league-of-ireland-premier-division/startseite/wettbewerb/IR1"),
    ("Iceland Besta deild", "https://www.transfermarkt.com/besta-deild/startseite/wettbewerb/IS1"),
    ("Northern Ireland Premiership","https://www.transfermarkt.com/premiership/startseite/wettbewerb/NIR1"), 
    ("Estonia Premium Liiga", "https://www.transfermarkt.com/premium-liiga/startseite/wettbewerb/EST1"),
    ("Montenegro Meridianbet 1. CFL","https://www.transfermarkt.com/meridianbet-1-cfl/startseite/wettbewerb/MNE1"), 
    ("Luxembourg BGL Ligue", "https://www.transfermarkt.com/bgl-ligue/startseite/wettbewerb/LUX1"),
    ("Andorra Primera Divisió", "https://www.transfermarkt.com/primera-divisio/startseite/wettbewerb/AND1"),
    ("Faroe Islands Betri-deildin", "https://www.transfermarkt.com/betri-deildin/startseite/wettbewerb/FARO"),
    ("Wales Cymru Premier", "https://www.transfermarkt.com/cymru-premier/startseite/wettbewerb/WAL1"),
    ("Gibraltar Gibraltar Football League","https://www.transfermarkt.com/gibraltar-football-league/startseite/wettbewerb/GI1"), 
    ("Sanmarino Camp. Sammarinese", "https://www.transfermarkt.com/campionato-sammarinese/startseite/wettbewerb/SMR1"),
    ("Malta Gozo League","https://www.transfermarkt.com/gozo-football-league-first-division/startseite/wettbewerb/GZO1"),
    ("Brazil Série A","https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1"),
    ("Argentina Liga Profesional","https://www.transfermarkt.com/professional-football-league/startseite/wettbewerb/AR1N"), 
    ("Colombia Liga Dimayor I","https://www.transfermarkt.com/liga-dimayor-apertura/startseite/wettbewerb/COLP"), 
    ("Chile Primera División","https://www.transfermarkt.com/primera-division-de-chile/startseite/wettbewerb/CLPD"), 
    ("Ecuador Serie A Primera Etapa","https://www.transfermarkt.com/ligapro-serie-a-primera-etapa/startseite/wettbewerb/EL1A"), 
    ("Uruguay Primera División Apertura","https://www.transfermarkt.com/primera-division-apertura/startseite/wettbewerb/URU1"), 
    ("Peru Liga 1 Apertura", "https://www.transfermarkt.com/liga-1-apertura/startseite/wettbewerb/TDeA"),
    ("Paraguay Primera División Apertura", "https://www.transfermarkt.com/primera-division-apertura/startseite/wettbewerb/PR1A"),
    ("Bolivia División Profesional Apertura","https://www.transfermarkt.com/division-profesional-apertura/startseite/wettbewerb/B1AP"),
    ("Venezuela Liga FUTVE Apertura","https://www.transfermarkt.com/liga-futve-apertura/startseite/wettbewerb/VZ1A"),
    ("United States MLS", "https://www.transfermarkt.com/major-league-soccer/startseite/wettbewerb/MLS1"),
    ("Mexico Liga MX Apertura", "https://www.transfermarkt.com/liga-mx-apertura/startseite/wettbewerb/MEXA"),
    ("Costa Rica Primera División Apertura", "https://www.transfermarkt.com/primera-division-apertura/startseite/wettbewerb/CRPD"),
    ("Honduras Liga Nacional Apertura", "https://www.transfermarkt.com/liga-nacional-apertura/startseite/wettbewerb/HO1A"),
    ("Guatemala Liga Guate Apertura", "https://www.transfermarkt.com/liga-guate-apertura/startseite/wettbewerb/GU1A"),
    ("El Salvador Primera División Apertura", "https://www.transfermarkt.com/primera-division-apertura/startseite/wettbewerb/SL1A"),
    ("Canada CanPL", "https://www.transfermarkt.com/canadian-premier-league/startseite/wettbewerb/CDN1"),
    ("Panama Liga Panameña Apertura", "https://www.transfermarkt.com/liga-panamena-de-futbol-apertura/startseite/wettbewerb/PN1A"),
    ("Nicaragua Liga Primera Apertura", "https://www.transfermarkt.com/liga-primera-de-nicaragua-apertura/startseite/wettbewerb/NC1A"),
    ("Dominican Republic Liga Dominicana de Fútbol", "https://www.transfermarkt.com/liga-dominicana-de-futbol/startseite/wettbewerb/DOM1"),
    ("Jamaica Jamaica Premier League","https://www.transfermarkt.com/jamaica-premier-league/startseite/wettbewerb/JPL1"),
    ("South Africa Betway Premiership", "https://www.transfermarkt.com/betway-premiership/startseite/wettbewerb/SFA1"),
    ("Egypt Premier League", "https://www.transfermarkt.com/egyptian-premier-league/startseite/wettbewerb/EGY1"),
    ("Morocco Botola Pro Inwi", "https://www.transfermarkt.com/botola-pro-inwi/startseite/wettbewerb/MAR1"),
    ("Argelia Ligue Professionnelle 1", "https://www.transfermarkt.com/ligue-professionnelle-1/startseite/wettbewerb/ALG1"),
    ("Tunisia Ligue I Pro", "https://www.transfermarkt.com/ligue-professionnelle-1/startseite/wettbewerb/TUN1"),
    ("Ghana Premier League", "https://www.transfermarkt.com/ghana-premier-league/startseite/wettbewerb/GHPL"),
    ("Angola Girabola", "https://www.transfermarkt.com/girabola/startseite/wettbewerb/AN1L"),
    ("Mozambique Moçambola", "https://www.transfermarkt.com/mocambola/startseite/wettbewerb/MO1L"),
    ("Uganda Premier League", "https://www.transfermarkt.com/uganda-premier-league/startseite/wettbewerb/UGL1"),
    ("Nigeria NPFL", "https://www.transfermarkt.com/nigeria-professional-football-league/startseite/wettbewerb/NPFL"),
    ("Ethiopia Premier League","https://www.transfermarkt.com/ethiopian-premier-league/startseite/wettbewerb/ETP1"),
    ("Saudi Arabia Saudi Pro League","https://www.transfermarkt.com/saudi-pro-league/startseite/wettbewerb/SA1"),
    ("United Araba Emirates UAE Pro League", "https://www.transfermarkt.com/uae-pro-league/startseite/wettbewerb/UAE1"),
    ("Qatar Stars League", "https://www.transfermarkt.com/qatar-stars-league/startseite/wettbewerb/QSL"),
    ("Japan J1 League","https://www.transfermarkt.com/j1-league/startseite/wettbewerb/JAP1"),
    ("Korea, South K League 1", "https://www.transfermarkt.com/k-league-1/startseite/wettbewerb/RSK1"),
    ("China Super League","https://www.transfermarkt.com/chinese-super-league/startseite/wettbewerb/CSL"),
    ("Iran Persian Gulf Pro League", "https://www.transfermarkt.com/persian-gulf-pro-league/startseite/wettbewerb/IRN1"),
    ("Uzbekistan Superliga", "https://www.transfermarkt.com/ozbekiston-superligasi/startseite/wettbewerb/UZ1"),
    ("Thailand Thai League","https://www.transfermarkt.com/thai-league/startseite/wettbewerb/THA1"), 
    ("Indonesia Liga 1","https://www.transfermarkt.com/liga-1-indonesia/startseite/wettbewerb/IN1L"), 
    ("India Indian Super League", "https://www.transfermarkt.com/indian-super-league/startseite/wettbewerb/IND1"),
    ("Malaysia Super League","https://www.transfermarkt.com/malaysia-super-league/startseite/wettbewerb/MYS1"), 
    ("Vietnam V.League 1", "https://www.transfermarkt.com/v-league-1/startseite/wettbewerb/VIE1"),
    ("Oman Oman Pro League", "https://www.transfermarkt.com/oman-professional-league/startseite/wettbewerb/OM1L"),
    ("Lebanon Leb. Premier League","https://www.transfermarkt.com/lebanese-premier-league/startseite/wettbewerb/LIB1"), 
    ("Hongkong Hong Kong Premier League", "https://www.transfermarkt.com/hong-kong-premier-league/startseite/wettbewerb/HGKG"),
    ("Singapore Premier League", "https://www.transfermarkt.com/singapore-premier-league/startseite/wettbewerb/SIN1"),
    ("Bangladesh Bangladesh PL", "https://www.transfermarkt.com/bangladesh-premier-league/startseite/wettbewerb/BGD1"),
    ("Cambodia C. Premier League", "https://www.transfermarkt.com/cambodian-premier-league/startseite/wettbewerb/KHM1"),
    ("Philippines PFL", "https://www.transfermarkt.com/philippines-football-league/startseite/wettbewerb/PFL1"),
    ("Tajikistan Vysshaya Liga","https://www.transfermarkt.com/vysshaya-liga/startseite/wettbewerb/TAD1"), 
    ("Chinese Taipei Football Premier League", "https://www.transfermarkt.com/taiwan-football-premier-league/startseite/wettbewerb/TFPL"),
    ("Myanmar National League","https://www.transfermarkt.com/myanmar-national-league/startseite/wettbewerb/MYA1"),
    ("Laos Lao League 1","https://www.transfermarkt.com/lao-league-1/startseite/wettbewerb/LAO1"),
    ("Australia A-League Men", "https://www.transfermarkt.com/a-league-men/startseite/wettbewerb/AUS1"),
    ("New Zeland National League - North", "https://www.transfermarkt.com/national-league-north/startseite/wettbewerb/NNL1"),
    ("New Zeland National League Championship","https://www.transfermarkt.com/national-league-championship/startseite/wettbewerb/NZNL"),
    ("New Zeland National League - Central", "https://www.transfermarkt.com/national-league-central/startseite/wettbewerb/NCL1"),
    ("Fiji Fiji Premier League", "https://www.transfermarkt.com/fiji-premier-league/startseite/wettbewerb/FIJ1"),
    ("New Zeland National League - South","https://www.transfermarkt.com/national-league-south/startseite/wettbewerb/NSL1")
    
    # Add more leagues here...
]


def extract_league_name(club_url, headers):
    response = requests.get(club_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Look for the span with class 'data-header__club' which contains the league information
        league_span = soup.find('span', class_='data-header__club')
        if league_span:
            league_link = league_span.find('a')
            if league_link:
                # Extract the league name
                league_name = league_link.get_text(strip=True)
                return league_name
    return 'N/A'


def extract_player_data_refined_fix(soup, headers):
    # Locate the main header for the player name
    name_tag = soup.find('h1', class_='data-header__headline-wrapper')
    
    if name_tag:
        # Get the full text inside the h1, including parts inside <strong> tags
        raw_name = ''.join([part.get_text(strip=True) if part.name else part for part in name_tag.contents])
        raw_name = raw_name.strip()

        # Handle the case where the name includes the jersey number (e.g., "#7 Cristiano Ronaldo")
        jersey_number = None
        jersey_match = re.match(r'#(\d+)\s*(.+)', raw_name)
        if jersey_match:
            jersey_number = jersey_match.group(1)
            name = jersey_match.group(2).strip()
        else:
            name = raw_name
    else:
        name = 'N/A'
        jersey_number = None

    # Process other player details (birthdate, death date, etc.) as before
    birth_info = soup.find('span', itemprop='birthDate')
    if birth_info:
        birth_date_raw = birth_info.get_text(strip=True).split('(')[0].strip()
        try:
            birth_date = datetime.strptime(birth_date_raw, "%b %d, %Y").strftime("%m/%d/%Y")
        except ValueError:
            birth_date = 'N/A'
        age_match = re.search(r'\((\d+)\)', birth_info.get_text(strip=True))
        age = age_match.group(1) if age_match else 'N/A'
    else:
        birth_date = 'N/A'
        age = 'N/A'

    death_tag = soup.find('span', itemprop='deathDate')
    death_date = death_tag.get_text(strip=True) if death_tag else 'N/A'
    if death_date != 'N/A':
        age_match = re.search(r'\((\d+)\)', death_date)
        age = age_match.group(1) if age_match else 'N/A'
        death_date_raw = death_date.split('(')[0].strip()
        try:
            death_date_formatted = datetime.strptime(death_date_raw, "%d.%m.%Y").strftime("%m/%d/%Y")
        except ValueError:
            death_date_formatted = 'N/A'
    else:
        death_date_formatted = 'N/A'

    citizenship_tag = soup.find('span', itemprop='nationality')
    citizenship = citizenship_tag.get_text(strip=True) if citizenship_tag else 'N/A'

    img_tag = citizenship_tag.find('img') if citizenship_tag else None
    img_src = img_tag['src'] if img_tag else 'N/A'

    # Modify the URL as needed
    if img_src != 'N/A':
        # Replace 'tiny' with 'head' in the URL
        nat_url = img_src.replace('/tiny/', '/head/')
    else:
        nat_url = 'N/A'

    position_tag = soup.find('dd', class_='detail-position__position')
    position = position_tag.get_text(strip=True) if position_tag else 'N/A'

    height_tag = soup.find(text=re.compile(r'Height:')).find_next('span') if soup.find(text=re.compile(r'Height:')) else None
    height_raw = height_tag.get_text(strip=True) if height_tag else 'N/A'
    if height_raw != 'N/A' and 'm' in height_raw:
        height = int(float(height_raw.replace('m', '').replace(',', '.').strip()) * 100)
    else:
        height = 'N/A'

    foot_label = soup.find(text=re.compile(r'Foot:')).find_next('span') if soup.find(text=re.compile(r'Foot:')) else None
    foot = foot_label.get_text(strip=True) if foot_label else 'N/A'

    current_club_info = soup.find(text=re.compile(r'Current club:')).find_next('a') if soup.find(text=re.compile(r'Current club:')) else None
    current_club_img = current_club_info.find('img') if current_club_info else None
    if current_club_img and current_club_img.has_attr('alt'):
        current_club = current_club_img['alt']
    elif current_club_info:
        current_club = current_club_info.get_text(strip=True)
    else:
        current_club = 'N/A'

    club_url = "https://www.transfermarkt.com" + current_club_info['href'] if current_club_info else None
    league = extract_league_name(club_url, headers) if club_url else 'N/A'

    # Print player data
    player_data = {
        'Player Name': name,
        'Jersey Number': jersey_number,
        'Birthday': birth_date,  # Ensure this is treated as a string
        'Age': age,
        'Death Date': death_date_formatted,
        'Nationality 1': citizenship,
        'Position': position,
        'Height': height,
        'Foot': foot,
        'Club Name': current_club,
        'nat_url': nat_url

    }
    print(player_data)  # Print player data

    return player_data

# Read the Excel file
df = pd.read_excel('players.xlsx')

print("Columns in DataFrame:", df.columns)

# Strip any leading/trailing spaces in the column names (if any)
df.columns = df.columns.str.strip()

# Check for 'Birthday' and 'Death Date' columns, create 'Death Date' if not present
if 'Death Date' not in df.columns:
    df['Death Date'] = None  # Create an empty 'Death Date' column

# Ensure 'Foot' column is treated as a string
if 'Foot' in df.columns:
    df['Foot'] = df['Foot'].astype(str)

# Convert 'Birthday' and 'Death Date' columns to object (string) type if they exist
df = df.astype({'Birthday': 'object', 'Death Date': 'object'}, errors='ignore')

# Headers to mimic browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
}

# Initialize a counter to track progress and save every 100 players
counter = 0

# Ensure 'Foot' column is treated as a string if it exists
if 'Foot' in df.columns:
    df['Foot'] = df['Foot'].astype(str)

# Iterate over rows where 'Player URL' is not empty
for index, row in df[df['Player URL'].notna()].iterrows():
    player_url = row['Player URL']
    print(f"Processing: {player_url}")
    
    # Send request
    response = requests.get(player_url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        player_data = extract_player_data_refined_fix(soup, headers)
        
        # Store the extracted data back into the DataFrame
        df.at[index, 'Player Name'] = player_data['Player Name']
        df.at[index, 'Jersey Number'] = player_data['Jersey Number']
        df.at[index, 'Birthday'] = str(player_data['Birthday'])  # Ensuring birthday is saved as a string
        df.at[index, 'Age'] = player_data['Age']
        df.at[index, 'Death Date'] = str(player_data['Death Date'])  # Ensuring death date is saved as a string
        df.at[index, 'Nationality 1'] = player_data['Nationality 1']
        df.at[index, 'Position'] = player_data['Position']
        
        # Handle 'Height (cm)' to ensure it's either a valid numeric value or NaN
        height = player_data['Height']
        df.at[index, 'Height'] = height if isinstance(height, int) else np.nan
        
        df.at[index, 'Foot'] = player_data['Foot']
        df.at[index, 'Club Name'] = player_data['Current Club']
        df.at[index, 'League'] = player_data['League']

    # Increment the counter
    counter += 1

    # Every 100 players, save the progress
    if counter % 100 == 0:
        df.to_excel(f'updated_players_{counter}.xlsx', index=False)
        print(f"Processed {counter} players. Progress saved to 'updated_players_{counter}.xlsx'.")

# Save the final updated DataFrame after all players are processed
df.to_excel('updated_players_final.xlsx', index=False)
print("Player data extraction complete. Final data saved to 'updated_players_final.xlsx'")

In [45]:
import pandas as pd

# Load the Excel file
file_path = 'updated_players_final.xlsx'
df = pd.read_excel(file_path)

# Define the nationality to continent mapping logic
def get_continent(nat):
    europe = ['England', 'Italy', 'Germany', 'Spain', 'France', 'Belgium', 'Portugal', 'Denmark', 'Russia', 'Switzerland', 
              'Netherlands', 'Türkiye', 'Austria', 'Croatia', 'Poland', 'Norway', 'Sweden', 'Cyprus', 'Czech Republic', 
              'Hungary', 'Greece', 'Romania', 'Scotland', 'Slovenia', 'Ukraine', 'Azerbaijan', 'Slovakia', 'Bulgaria', 
              'Serbia', 'Bosnia-Herzegovina', 'Kazakhstan', 'Finland', 'Georgia', 'Albania', 'Belarus', 'Armenia', 
              'Ireland', 'Iceland', 'Kosovo', 'North Macedonia', 'Moldova', 'Lithuania', 'Latvia', 'Malta', 'Luxembourg', 
              'Montenegro', 'Faroe Islands', 'Northern Ireland', 'Estonia', 'Andorra', 'Gibraltar', 'San Marino']
    
    asia = ['Israel', 'Japan', 'Saudi Arabia', 'Iran', 'United Arab Emirates', 'Iraq', 'China', 'Uzbekistan', 'Kuwait', 
            'Jordan', 'Thailand', 'Bahrain', 'Syria', 'Qatar', 'Oman', 'Malaysia', 'Vietnam', 'Indonesia', 'Palestine', 
            'Lebanon', 'India', 'Nepal', 'Singapore', 'Hongkong', 'Yemen', 'Turkmenistan', 'Philippines', 'Tajikistan', 
            'Kyrgyzstan', 'Myanmar', 'Cambodia', 'Chinese Taipei', 'Bangladesh', 'Maldives', 'Mongolia', 'Macao', 
            'Brunei Darussalam', 'Guam', 'Laos', 'Bhutan']
    
    africa = ['Algeria', 'Egypt', 'South Africa', 'Tunisia', 'Ghana', 'Nigeria', 'Zambia', 'Cote d\'Ivoire', 'Senegal', 
              'Réunion', 'Mali', 'Tanzania', 'Zimbabwe', 'Mozambique', 'Libya', 'Angola', 'Burkina Faso', 'Togo', 'Guinea', 
              'Uganda', 'Ethiopia', 'DR Congo', 'Kenya', 'Cape Verde', 'Congo', 'Mauritania', 'Cameroon', 'The Gambia', 
              'Burundi', 'Rwanda', 'Madagascar', 'Botsuana', 'Eswatini', 'Benin', 'Lesotho', 'Malawi', 'Mauritius', 
              'Liberia', 'Sao Tome and Principe', 'Sierra Leone', 'Niger', 'Somalia']
    
    north_america = ['United States', 'Mexico', 'Canada']
    
    central_america = ['Costa Rica', 'Panama', 'Guatemala', 'Honduras', 'El Salvador', 'Jamaica', 'Nicaragua', 'Dominican Republic', 
                       'Grenada', 'Haiti', 'Suriname', 'Antigua and Barbuda', 'French Guiana', 'Martinique', 'Puerto Rico', 'Aruba', 
                       'St. Kitts & Nevis', 'British Virgin Islands', 'Trinidad and Tobago', 'Guadeloupe', 'Turks- and Caicosinseln', 
                       'Belize', 'Curacao', 'Cayman Islands', 'Cuba', 'Bermuda', 'Guyana', 'Barbados']
    
    south_america = ['Brazil', 'Argentina', 'Ecuador', 'Colombia', 'Paraguay', 'Chile', 'Uruguay', 'Bolivia', 'Peru', 'Venezuela']
    
    oceania = ['Australia', 'New Zealand', 'Neukaledonien', 'Fiji', 'Papua New Guinea', 'Solomon Islands', 'Cookinseln', 'Tahiti']

    if nat in europe:
        return 'Europe'
    elif nat in asia:
        return 'Asia'
    elif nat in africa:
        return 'Africa'
    elif nat in north_america:
        return 'North America'
    elif nat in central_america:
        return 'Central America'
    elif nat in south_america:
        return 'South America'
    elif nat in oceania:
        return 'Oceania'
    else:
        return ''

# Apply the continent mapping based on 'Nationality 1' column
df['Continent'] = df['Nationality 1'].apply(get_continent)

# Save the updated DataFrame back to Excel
df.to_excel('updated_players_final_with_continent.xlsx', index=False)

print("Continent column has been successfully updated!")


Continent column has been successfully updated!
